In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random
import pandas_profiling
import sidetable
import xgboost as xgb
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, roc_auc_score, make_scorer, confusion_matrix, classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.utils.class_weight import compute_class_weight
from sklearn.impute import KNNImputer

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier, Pool
import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

pd.set_option('display.max_columns', 300)

from IPython.core.display import display, HTML # расширяем блокнот
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_columns', None) # выводим все колонки

from IPython.core.interactiveshell import InteractiveShell # выводим все результаты из ячейки
InteractiveShell.ast_node_interactivity = "all"

In [16]:
x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv', index_col=0)
x_test = pd.read_csv('x_test.csv')

print(x_train.shape, x_test.shape, y_train.shape)

(2838, 68) (939, 68) (2838, 1)


# Обучаем модели

In [3]:
def hyperopt_xgb_score(params):
    clf = XGBClassifier(**params)
    # усреднение по 3ем фолдам, для уменьшения влияния стахостичности
    # для ускорения можно использовать train_test_split один раз
    current_score = cross_val_score(clf, x_train, y_train, cv=3).mean()
    print(current_score, params)
    return -current_score

In [4]:
space_xgb2 = {
            'n_estimators': hp.choice('n_estimators', range(50, 500)),
            'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
            'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
            'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
            'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
            'eval_metric': 'auc',
            'objective': 'binary:logistic',
            # Increase this number if you have more cores. Otherwise, remove it and it will default
            # to the maxium number.
            'nthread': 4,
            'booster': 'gbtree',
            'tree_method': 'exact',
            'silent': 1
        }

In [5]:
best = fmin(fn=hyperopt_xgb_score, space=space_xgb2, algo=tpe.suggest, max_evals=5)
print('best:')
print(best)

nan                                                  
{'booster': 'gbtree', 'colsample_bytree': 0.8500000000000001, 'eta': 0.225, 'eval_metric': 'auc', 'gamma': 0.9500000000000001, 'max_depth': 4, 'min_child_weight': 5.0, 'n_estimators': 65, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.9500000000000001, 'tree_method': 'exact'}
nan                                                          
{'booster': 'gbtree', 'colsample_bytree': 0.8500000000000001, 'eta': 0.35000000000000003, 'eval_metric': 'auc', 'gamma': 0.8500000000000001, 'max_depth': 8, 'min_child_weight': 1.0, 'n_estimators': 457, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.75, 'tree_method': 'exact'}
 40%|████      | 2/5 [00:00<00:00,  3.58trial/s, best loss=?]

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/antonskarednov/opt/anaconda3/lib/python3.7/sit

nan                                                          
{'booster': 'gbtree', 'colsample_bytree': 0.7000000000000001, 'eta': 0.275, 'eval_metric': 'auc', 'gamma': 0.8, 'max_depth': 8, 'min_child_weight': 5.0, 'n_estimators': 155, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.8500000000000001, 'tree_method': 'exact'}
nan                                                          
{'booster': 'gbtree', 'colsample_bytree': 1.0, 'eta': 0.375, 'eval_metric': 'auc', 'gamma': 0.8, 'max_depth': 6, 'min_child_weight': 1.0, 'n_estimators': 81, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.5, 'tree_method': 'exact'}
 80%|████████  | 4/5 [00:00<00:00,  4.59trial/s, best loss=?]

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/antonskarednov/opt/anaconda3/lib/python3.7/sit

nan                                                          
{'booster': 'gbtree', 'colsample_bytree': 0.9, 'eta': 0.05, 'eval_metric': 'auc', 'gamma': 0.65, 'max_depth': 9, 'min_child_weight': 6.0, 'n_estimators': 399, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.8500000000000001, 'tree_method': 'exact'}
100%|██████████| 5/5 [00:00<00:00,  5.24trial/s, best loss=?]


/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/antonskarednov/opt/anaconda3/lib/python3.7/sit

AllTrialsFailed: 

In [6]:
xgb = XGBClassifier(
learning_rate =0.1,
 n_estimators=15,
 max_depth=9,
 min_child_weight=5,
 gamma=0.65,
 subsample=0.9,
 colsample_bytree=0.75,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27
)

In [ ]:
xgb.fit(x_train, y_train)

In [49]:
preds = xgb.predict(x_test)

In [46]:
preds = pd.DataFrame(preds)

In [47]:
preds.value_counts()

4     503
7     111
9      72
10     66
8      36
5      28
14     23
12     19
3      18
16     17
2      14
21     12
11      9
1       9
22      1
17      1
dtype: int64

Основные параметры модели:
n_estimators — число "деревьев" в "случайном лесу" (по дефолту – 10)
criterion — функция, которая измеряет качество разбиения ветки дерева (по дефолту — mse , так же можно выбрать "mae")
max_features — число признаков, по которым ищется разбиение. Вы можете указать конкретное число или процент признаков, либо выбрать из доступных значений: auto (все признаки), sqrt, log2. По дефолту стоит auto.
max_depth — максимальная глубина дерева (по дефолту глубина не ограничена)
min_samples_split — минимальное количество объектов, необходимое для разделения внутреннего узла. Можно задать числом или процентом от общего числа объектов (по дефолту — 2)
min_samples_leaf — минимальное число объектов в листе. Можно задать числом или процентом от общего числа объектов (по дефолту — 1)
min_weight_fraction_leaf — минимальная взвешенная доля от общей суммы весов (всех входных объектов) должна быть в листе (по дефолту имеют одинаковый вес)
max_leaf_nodes — максимальное количество листьев (по дефолту нет ограничения)
min_impurity_split — порог для остановки наращивания дерева (по дефолту 1е-7)
bootstrap— использование для построения деревьев подвыборки с возвращением (по дефолту True)
oob_score — использовать ли out-of-bag объекты для оценки R2 (по дефолту False)
n_jobs — количество ядер процессора (по дефолту 1, если поставить -1, то будут использоваться все ядра)
random_state — фиксация состояния генератора случайных чисел (по дефолту - None, если хотите воспроизводимые результаты, то нужно указать любое число типа int)
verbose — вывод логов по построению деревьев (по дефолту 0)
warm_start — использует уже натренированую модель и добавляет деревьев в ансамбль (по дефолту False)
max_features — число признаков для выбора расщепления.
max_depth — максимальная глубина деревьев.
min_samples_split — минимальное число объектов, необходимое для того, чтобы узел дерева мог бы расщепиться.
min_samples_leaf — минимальное число объектов в листьях.

In [19]:
RFC = RandomForestClassifier(random_state=12344,
                               n_estimators=500, 
                               max_depth=30, 
                               max_features = 'auto',
                               class_weight='balanced'
                            )
RFC.fit(x_train, y_train)
preds_valid = RFC.predict(x_test)

#print(f1_score(y_test, preds_valid, average='weighted'))

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


RandomForestClassifier(class_weight='balanced', max_depth=30, n_estimators=500,
                       random_state=12344)

In [20]:
preds = RFC.predict(x_test) 

In [133]:
preds = pd.DataFrame(preds)
preds.value_counts() #7184

4     497
7      98
9      82
3      59
8      51
10     49
5      24
21     22
12     19
16     12
14     12
2       3
17      3
28      2
23      2
6       1
15      1
18      1
11      1
dtype: int64

In [50]:
from sklearn.utils.class_weight import compute_class_weight
CLASS_WEIGHTS = compute_class_weight('balanced', classes = np.unique(y_train), y = np.ravel(y_train))

CLASS_WEIGHTS

array([3.00264550e+00, 2.25198413e+00, 3.11813187e-01, 7.72844886e-02,
       7.50661376e-01, 6.75595238e+00, 4.20059215e-01, 1.21002132e+00,
       5.44103547e-01, 1.65451895e+00, 2.19111969e+00, 1.24725275e+00,
       3.00264550e+00, 1.15816327e+00, 1.15816327e+01, 1.72492401e+00,
       4.76890756e+00, 8.10714286e+01, 8.10714286e+01, 1.35119048e+01,
       3.86054422e+00, 1.15816327e+01, 2.02678571e+01, 2.02678571e+01,
       4.05357143e+01, 8.10714286e+01, 8.10714286e+01, 4.05357143e+01])

In [45]:
X_train, X_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=12345)

In [46]:
def final_pred(model, features, target, test):
   
    model.fit(features, target)
    
    pred_final = pd.DataFrame(model.predict(test), columns=['Predicted'])
 
    return pred_final

    
def val_pred(model, X_train, y_train, X_val, y_val):
    
    model.fit(X_train, y_train)
    
    pred_train = model.predict(X_train)
    pred_valid = model.predict(X_val)
    
    f1_train = f1_score(y_train, pred_train, average='weighted').round(3)
    f1_test = f1_score(y_val, pred_valid, average='weighted').round(3)
    cl_rep = classification_report(y_val, pred_valid)
    
    cf = confusion_matrix(y_val, pred_valid)
    
    
    print()
    print('train', f1_train)
    print('test', f1_test)
    print()
    print(cf)
    print()
    print(cl_rep)

In [47]:
from catboost import CatBoostClassifier, Pool

In [51]:
# Базовый CATBoost
cbc = CatBoostClassifier(random_state=42, verbose = 0, class_weights=CLASS_WEIGHTS)
val_pred(cbc, X_train, y_train, X_valid, y_valid)


train 0.995
test 0.897

[[  7   1   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0  11   1   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0
    0   0   0   0   0   0]
 [  0   0  76   0   0   0   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0 248   1   0   0   0   0   0   1   0   0   1   1   0   0   0
    0   0   0   0   0   0]
 [  0   0   1   0  22   0   0   1   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   2   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0  11   0   2   0  39   0   0   1   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  2   1   0   0   4   0   0   9   0   0   0   0   0   1   0   0   0   0
    0   0   1   0   0   0]
 [  0   1   0   0   0   0   0   0  36   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   1   0   0   9   0   1   0   0   0   0 

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
cbc1 = CatBoostClassifier(
              l2_leaf_reg = 0.1,                        
              verbose=0, 
              class_weights=CLASS_WEIGHTS,
              random_seed=42)
val_pred(cbc1, X_train, y_train, X_valid, y_valid)


train 1.0
test 0.908

[[  8   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0  11   1   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0
    0   0   0   0   0]
 [  0   0  76   0   0   0   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0 251   0   0   0   0   0   0   1   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   1   1  22   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   2   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   8   0   2   0  42   0   0   1   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  2   1   0   1   4   0   0   9   0   0   0   0   0   0   0   0   0   0
    0   1   0   0   0]
 [  0   0   0   0   0   0   0   0  37   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   1   0   0   0   1   0   0   9   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0  

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [80]:
vc = VotingClassifier([('clf1', cbc), ('clf2', cbc1)], voting='soft')
val_pred(vc, X_train, y_train, X_valid, y_valid)


train 0.999
test 0.911

[[  7   1   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0  11   1   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0
    0   0   0   0   0]
 [  0   0  76   0   0   0   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0 251   0   0   0   0   0   0   1   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   1   1  21   0   0   1   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   2   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   7   0   2   0  43   0   0   1   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  2   1   0   1   3   0   0  11   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   1   0   0   0   0   0   0  36   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   1   0   0   0   1   0   0   8   0   1   0   0   0   0   0   0
    0   0   0   0   0]
 [  0

In [81]:
pred_final = final_pred(vc, X_train, y_train, x_test)

In [224]:
pred_final1 = final_pred(cbc, X_train, y_train, x_test)

In [220]:
pred_final2 = final_pred(cbc1, X_train, y_train, x_test)

In [82]:
pred_final.value_counts()

Predicted
4            459
9             80
3             70
7             62
10            52
21            45
8             45
12            27
5             24
14            14
16            14
1             12
11            10
2              8
17             4
15             3
28             3
27             1
18             1
20             1
6              1
23             1
25             1
26             1
dtype: int64

In [221]:
pred_final2.value_counts()

Predicted
4            461
9             81
3             66
10            58
7             58
8             53
21            48
12            28
14            17
5             16
16            13
1             10
11             6
17             5
2              4
28             2
15             2
20             2
23             2
13             2
27             1
18             1
6              1
25             1
26             1
dtype: int64

In [225]:
pred_final1.value_counts()

Predicted
4            456
9             83
7             70
10            59
3             55
21            49
8             47
12            25
5             16
14            14
2             13
16            12
11             8
1              5
17             5
13             4
15             3
6              3
20             3
28             3
23             2
27             1
18             1
25             1
26             1
dtype: int64

In [83]:
pred_final.rename(columns={'Predicted': 'Culture'}, inplace=True)

In [84]:
pred_final.to_csv('preds5.csv', index=False)